weatherAUS 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = Path(root_path) / "configs" / "weatherAUS_bttwd.yaml"
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-12-15 19:12:26】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\weatherAUS_bttwd.yaml
【INFO】【2025-12-15 19:12:31】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-12-15 19:12:31】【配置-数据】数据集=weatherAUS, k折=5, 目标列=RainTomorrow, 正类="Yes"
【INFO】【2025-12-15 19:12:31】【配置-BTTWD】阈值模式=bucket_wise, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-12-15 19:12:31】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-12-15 19:12:31】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（流失/未流失）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='流失 vs 未流失比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始标签列的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是原始标签列
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Telco Churn 原始数据加载与基本统计完成。')


【INFO】【2025-12-15 19:12:31】【数据加载】文本表格 ..\data\weather\weatherAUS.csv 已读取，样本数=145460，列数=23


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-12-12 21:27:23】【预处理】连续特征=7个，类别特征=9个
【INFO】【2025-12-12 21:27:23】【预处理】编码后维度=45
【INFO】【2025-12-12 21:27:23】【预处理】编码特征维度=45，样本数=25000
【INFO】【2025-12-12 21:27:23】【步骤3摘要】特征预处理完成：连续=7，类别=9，编码维度=45。


In [ ]:
# 步骤4：构建桶树并检查划分
feature_cols_for_bucket = [c for c in df_raw.columns if c != target_col_model]

bucket_tree = BucketTree(
    cfg['BTTWD']['bucket_levels'],
    feature_names=feature_cols_for_bucket
)

bucket_ids_full = bucket_tree.assign_buckets(df_raw[feature_cols_for_bucket])

bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']

bucket_df['pos_rate'] = (
    df_raw.groupby(bucket_ids_full)[target_col_model]
    .apply(lambda s: (s == 1).mean())
    .values
)
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')


【INFO】【2025-12-12 21:27:23】【桶树】已为样本生成桶ID，共 45 个组合


,bucket_id,count,pos_rate
0,L1_age=young_old|L2_medical_specialty=Others|L...,3671,0.333333
1,L1_age=young_old|L2_medical_specialty=Others|L...,2874,0.440860
2,L1_age=mid_age|L2_medical_specialty=Others|L3_...,2148,0.403636
3,L1_age=mid_age|L2_medical_specialty=Others|L3_...,1464,0.471429
4,L1_age=oldest|L2_medical_specialty=Others|L3_t...,1315,0.471299


【INFO】【2025-12-12 21:27:24】【步骤4摘要】桶树划分完成，共有 45 个叶子桶。


In [ ]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-12-12 21:27:24】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-12-12 21:27:24】【步骤5摘要】基线模型性能将作为后续对比基准。


In [ ]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始标签列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0, 1]), array([13246, 11754], dtype=int64))
原始标签列分布：
readmitted
0    13246
1    11754
Name: count, dtype: int64


In [ ]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-12-12 21:27:24】【基线-LogReg】使用决策阈值=0.200（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-12 21:27:25】【基线-LogReg】整体指标：AUC_mean=0.646, AUC_std=0.003, BAC_mean=0.501, BAC_std=0.000, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.640, F1_std=0.000, Kappa_mean=0.001, Kappa_std=0.001, MCC_mean=0.018, MCC_std=0.010, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.470, Precision_std=0.000, Recall_mean=1.000, Recall_std=0.000, Regret_mean=0.529, Regret_std=0.000
【INFO】【2025-12-12 21:27:25】【基线-RF】使用决策阈值=0.200（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-12 21:27:31】【基线-RF】整体指标：AUC_mean=0.639, AUC_std=0.005, BAC_mean=0.508, BAC_std=0.000, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.641, F1_std=0.001, Kappa_mean=0.015, Kappa_std=0.001, MCC_mean=0.057, MCC_std=0.004, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.474, Precision_std=0.000, Recall_mean=0.989, Recall_std=0.002, Regret_mean=0.531, Regret_std=0.002
【INFO】【2025-12-12 21:27:31】【基线-KNN】使用决策阈值=0.200（fixed 模式）


...

【INFO】【2025-12-12 21:28:28】【阈值】桶 ROOT（n_val=1199）使用本地阈值 α=0.2000, β=0.0000
[INFO][BT][2025-12-12 21:28:28] 桶 bucket_id=ROOT level=0：
    n_train=2799, n_val=1199,
    BAC=0.495, F1=0.659, AUC=0.621,
    Regret=0.508, BND_ratio=0.028, POS_coverage=0.972,
    Score(f1_regret_bnd )=0.137
【INFO】【2025-12-12 21:28:29】【阈值】桶 L1_age=mid_age 标记为弱桶，阈值将回退使用 ROOT 的阈值
[INFO][BT][2025-12-12 21:28:29] 桶 bucket_id=L1_age=mid_age level=1：
    n_train=780, n_val=334,
    BAC=0.439, F1=0.634, AUC=0.672,
    Regret=0.528, BND_ratio=0.210, POS_coverage=0.787,
    Score(f1_regret_bnd )=0.001
[INFO][BT][2025-12-12 21:28:29] 桶 bucket_id=L1_age=mid_age：
    parent_id=ROOT，parent_Score=0.137, bucket_Score=0.001,
    Gain=-0.136, is_weak=True
【INFO】【2025-12-12 21:28:29】【阈值】桶 L1_age=oldest 标记为弱桶，阈值将回退使用 ROOT 的阈值
[INFO][BT][2025-12-12 21:28:29] 桶 bucket_id=L1_age=oldest level=1：
    n_train=582, n_val=380,
    BAC=0.474, F1=0.634, AUC=0.600,
    Regret=0.534, BND_ratio=0.076, POS_coverage=0.924,
    Score(f1_regret

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [21:28:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-12-12 21:28:40】[BASELINE] 全局 XGB 模型训练完成
【INFO】【2025-12-12 21:28:40】[BASELINE] 阈值搜索开始
【INFO】【2025-12-12 21:28:40】[BASELINE] 最佳阈值找到: alpha=0.3000, beta=0.0000, regret=0.5172
【INFO】【2025-12-12 21:28:40】【桶树】已为样本生成桶ID，共 45 个组合
【INFO】【2025-12-12 21:28:40】[BASELINE] 测试集桶映射完成，共 45 个桶
【INFO】【2025-12-12 21:28:41】[BASELINE] 桶 L1_age=young_old|L2_medical_specialty=Others|L3_time_in_hospital=mid_stay: BAC=0.5153, Regret=0.5069, Precision=0.5065, Recall=0.9481
【INFO】【2025-12-12 21:28:41】[BASELINE] 桶 L1_age=young_old|L2_medical_specialty=Others|L3_time_in_hospital=long_stay: BAC=0.5370, Regret=0.4752, Precision=0.5540, Recall=0.9147
【INFO】【2025-12-12 21:28:41】[BASELINE] 桶 L1_age=mid_age|L2_medical_specialty=InternalMed|L3_time_in_hospital=short_stay: BAC=0.6635, Regret=0.5303, Precision=0.5370, Recall=0.7436
【INFO】【2025-12-12 21:28:41】[BASELINE] 桶 L1_age=mid_age|L2_medical_specialty=Others|L3_time_in_hospital=short_stay: BAC=0.5726, Regret=0.5859, Precision=0.4245, Recall=0.8491
【INFO】【20

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.487137,0.001709,0.955504,0.007202,0.645287,0.002922,0.531429,0.003141,0.649473,...,0.117409,0.013790,0.059628,0.005949,0.10504,0.032246,0.88756,0.031898,0.52256,0.001286
1,LogReg,0.470421,0.000270,0.999915,0.000170,0.639828,0.000223,0.500524,0.000428,0.646378,...,0.018265,0.010084,0.000985,0.000806,0.00000,0.000000,NaN,NaN,0.52936,0.000388
2,RandomForest,0.474191,0.000304,0.988684,0.001839,0.640964,0.000556,0.507931,0.000485,0.639255,...,0.057103,0.003986,0.014949,0.000916,0.00000,0.000000,NaN,NaN,0.53140,0.001876
3,KNN,0.492499,0.001811,0.843117,0.005035,0.621782,0.002280,0.536083,0.002764,0.584971,...,0.090894,0.006801,0.069386,0.005331,0.00000,0.000000,NaN,NaN,0.62976,0.006158
4,XGBoost,0.475988,0.000858,0.987409,0.000876,0.642334,0.000894,0.511408,0.001664,0.655854,...,0.073222,0.008968,0.021514,0.003141,0.00000,0.000000,NaN,NaN,0.52884,0.002271


【INFO】【2025-12-12 21:28:42】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [ ]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,is_weak,threshold_source_bucket,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,2799,1198,0.478742,0.484975,0.3,0.1,0.512521,...,False,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.470200
1,L1_age=young_old,L1,ROOT,1428,612,0.460784,0.475490,0.2,0.0,0.520425,...,False,L1_age=young_old,NaN,NaN,NaN,NaN,NaN,NaN,1,0.482394
2,L1_age=young_old|L2_medical_specialty=Others,L2,L1_age=young_old,4378,1847,0.481498,0.489442,0.2,0.1,0.506497,...,False,L1_age=young_old|L2_medical_specialty=Others,NaN,NaN,NaN,NaN,NaN,NaN,1,0.483855
3,L1_age=mid_age,L1,ROOT,787,396,0.443456,0.441919,0.3,0.1,0.527778,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.439239
4,L1_age=oldest,L1,ROOT,579,300,0.483592,0.473333,0.2,0.0,0.520000,...,False,L1_age=oldest,NaN,NaN,NaN,NaN,NaN,NaN,1,0.481570


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_24584\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-12-12 21:28:45】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [ ]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。')

【INFO】【2025-12-12 21:28:45】【步骤8】检查结果文件与图表。
['baseline_bucket_metrics.csv', 'bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-12-12 21:28:45】【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。
